# Task 1: Rating Prediction via Prompting

## Overview

This notebook implements a **star rating prediction system** for Yelp reviews using Large Language Models (LLMs) via LangChain and OpenAI.

### What This Notebook Evaluates
- **Zero-shot prompting capability**: Can an LLM accurately predict star ratings (1-5) from review text without fine-tuning?
- **Structured output reliability**: How well does the LLM adhere to a strict JSON schema?
- **Baseline performance**: Establishes Prompt V1 metrics for comparison with optimized prompts.

### High-Level Workflow
1. Load and sample the Yelp reviews dataset (stratified by star rating)
2. Define a structured output schema using Pydantic
3. Construct an LCEL (LangChain Expression Language) pipeline
4. Run batch predictions on the sampled dataset
5. Compute evaluation metrics (accuracy, JSON validity rate)
6. Persist results for reproducibility

---
## 1. Imports & Environment Setup

We use **modern LangChain APIs** (LCEL/Runnable pipelines) and **Pydantic v2** for structured output validation.

In [116]:
# Standard library imports
import os
import json
from typing import Optional, Tuple, Dict, Any

# Data handling
import pandas as pd

# LangChain - Modern imports (LCEL compatible)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

# Pydantic v2 for structured output
from pydantic import BaseModel, Field, field_validator

# Progress tracking
from tqdm import tqdm

print("✅ All imports successful")

✅ All imports successful


In [117]:
# Validate OpenAI API Key
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise EnvironmentError(
        "OPENAI_API_KEY environment variable is not set. "
        "Please set it before running this notebook:\n"
        "  Windows: set OPENAI_API_KEY=your-key-here\n"
        "  Linux/Mac: export OPENAI_API_KEY=your-key-here"
    )

# Mask the key for display
masked_key = OPENAI_API_KEY[:8] + "..." + OPENAI_API_KEY[-4:]
print(f"✅ OPENAI_API_KEY validated: {masked_key}")

✅ OPENAI_API_KEY validated: sk-proj-...tyIA


---
## 2. Dataset Loading & Sampling

We load the Yelp reviews dataset and perform **stratified sampling** to ensure balanced representation across all star ratings.

In [118]:
# Load the dataset
DATASET_PATH = "yelp.csv"

df_full = pd.read_csv(DATASET_PATH)

print(f"📊 Dataset loaded: {len(df_full):,} reviews")
print(f"📋 Columns: {df_full.columns.tolist()}")
print(f"\n⭐ Star distribution:")
print(df_full['stars'].value_counts().sort_index())

📊 Dataset loaded: 10,000 reviews
📋 Columns: ['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id', 'cool', 'useful', 'funny']

⭐ Star distribution:
stars
1     749
2     927
3    1461
4    3526
5    3337
Name: count, dtype: int64


In [119]:
# Stratified sampling: ~40 reviews per star rating = 200 total
SAMPLES_PER_STAR = 40
RANDOM_SEED = 42

def stratified_sample(df: pd.DataFrame, n_per_class: int, seed: int) -> pd.DataFrame:
    """
    Perform stratified sampling by star rating.
    
    Args:
        df: Source dataframe
        n_per_class: Number of samples per star rating
        seed: Random seed for reproducibility
    
    Returns:
        Stratified sample dataframe
    """
    samples = []
    for star in range(1, 6):
        star_df = df[df['stars'] == star]
        n_available = len(star_df)
        n_sample = min(n_per_class, n_available)
        samples.append(star_df.sample(n=n_sample, random_state=seed))
    
    return pd.concat(samples, ignore_index=True)

# Create stratified sample
df_sample = stratified_sample(df_full, SAMPLES_PER_STAR, RANDOM_SEED)

# Keep only required columns
df_sample = df_sample[['review_id', 'text', 'stars']].copy()

print(f"📊 Sampled dataset: {len(df_sample)} reviews")
print(f"\n⭐ Sampled star distribution:")
print(df_sample['stars'].value_counts().sort_index())

📊 Sampled dataset: 200 reviews

⭐ Sampled star distribution:
stars
1    40
2    40
3    40
4    40
5    40
Name: count, dtype: int64


In [120]:
# Data validation assertions

# Assert no null text values
assert df_sample['text'].notna().all(), "Found null text values in sample!"
print("✅ No null text values")

# Assert star values are in valid range [1, 5]
assert df_sample['stars'].isin([1, 2, 3, 4, 5]).all(), "Found invalid star values!"
print("✅ All star values are in range [1, 5]")

# Assert no duplicate review IDs
assert df_sample['review_id'].nunique() == len(df_sample), "Found duplicate review IDs!"
print("✅ All review IDs are unique")

# Assert minimum text length
assert (df_sample['text'].str.len() > 10).all(), "Found reviews with very short text!"
print("✅ All reviews have sufficient text length")

print(f"\n🎯 Dataset validation complete: {len(df_sample)} reviews ready for inference")

✅ No null text values
✅ All star values are in range [1, 5]
✅ All review IDs are unique
✅ All reviews have sufficient text length

🎯 Dataset validation complete: 200 reviews ready for inference


---
## 3. Output Schema Definition

We define a **Pydantic model** for structured LLM output. This ensures:
- Type safety (predicted_stars must be an integer)
- Value constraints (stars between 1-5)
- Required fields (explanation must be non-empty)

In [121]:
class RatingPrediction(BaseModel):
    """
    Structured output schema for rating prediction.
    
    Attributes:
        predicted_stars: The predicted star rating (1-5)
        explanation: Reasoning for the prediction
    """
    predicted_stars: int = Field(
        ...,
        description="Predicted star rating from 1 to 5",
        ge=1,
        le=5
    )
    explanation: str = Field(
        ...,
        description="Brief explanation for the predicted rating",
        min_length=1
    )
    
    @field_validator('predicted_stars')
    @classmethod
    def validate_stars_range(cls, v: int) -> int:
        """Ensure predicted_stars is within valid range."""
        if v < 1 or v > 5:
            raise ValueError(f"predicted_stars must be between 1 and 5, got {v}")
        return v
    
    @field_validator('explanation')
    @classmethod
    def validate_explanation_not_empty(cls, v: str) -> str:
        """Ensure explanation is not empty or whitespace only."""
        if not v or not v.strip():
            raise ValueError("explanation must be non-empty")
        return v.strip()

# Test the schema
test_prediction = RatingPrediction(predicted_stars=4, explanation="Great review with positive sentiment")
print(f"✅ Schema validated: {test_prediction.model_dump()}")

✅ Schema validated: {'predicted_stars': 4, 'explanation': 'Great review with positive sentiment'}


---
## 4. Output Parser

We use `PydanticOutputParser` to:
1. **Generate format instructions** for the LLM (tells it exactly how to structure output)
2. **Parse and validate** LLM responses against our schema

### Why Strict Parsing Matters
- **Reliability**: Ensures consistent, machine-readable output
- **Error Detection**: Catches malformed responses early
- **Type Safety**: Guarantees correct data types for downstream processing

In [122]:
# Initialize the output parser
output_parser = PydanticOutputParser(pydantic_object=RatingPrediction)

# Get format instructions for the prompt
format_instructions = output_parser.get_format_instructions()

print("📋 Format Instructions for LLM:")
print("-" * 50)
print(format_instructions)

📋 Format Instructions for LLM:
--------------------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Structured output schema for rating prediction.\n\nAttributes:\n    predicted_stars: The predicted star rating (1-5)\n    explanation: Reasoning for the prediction", "properties": {"predicted_stars": {"description": "Predicted star rating from 1 to 5", "maximum": 5, "minimum": 1, "title": "Predicted Stars", "type": "integer"}, "explanation": {"description": "Brief explanation for the predicted rating", "minLength": 1, "title": "Explanation", "typ

---
## 5. LLM Initialization

We use `ChatOpenAI` with:
- **temperature=0**: Deterministic outputs for reproducibility
- **gpt-4o-mini**: Cost-effective model with strong capabilities

In [123]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.2,
    api_key=OPENAI_API_KEY,
    max_retries=2,
    request_timeout=60
)

print(f"✅ LLM initialized: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")
print(f"   Max retries: {llm.max_retries}")

✅ LLM initialized: gpt-4.1-nano
   Temperature: 0.2
   Max retries: 2


---
## 6. Prompt Construction (Baseline - Prompt V1)

This is our **baseline prompt** without any optimization. It follows best practices:
- **System message**: Defines the role and task
- **Human message**: Contains the review text and format instructions

Future iterations can optimize this prompt for better accuracy.

In [124]:
# Define the prompt template
prompt_template1 = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are an expert sentiment analyst specializing in customer reviews.

Your task is to predict the star rating (1 to 5) of a Yelp review based on its text content.

Rating Guidelines:
- 1 star: Very negative experience, strong complaints
- 2 stars: Negative experience with some issues
- 3 stars: Mixed or neutral experience
- 4 stars: Positive experience with minor issues
- 5 stars: Excellent experience, highly positive

Analyze the sentiment, tone, and specific feedback in the review to make your prediction."""
    ),
    (
        "human",
        """Please analyze the following Yelp review and predict its star rating.

REVIEW:
{review_text}

---

{format_instructions}

Provide your prediction in the exact JSON format specified above."""
    )
])



prompt_template2 = ChatPromptTemplate.from_messages([("system","""
<SYSTEM>
You are an expert AI specialized in predicting Yelp star ratings (1-5) from review text and providing a brief, grounded explanation.
</SYSTEM>

<TASK>
Predict the most likely integer star rating (1-5) a Yelp user would give based solely on the review text.
Provide a concise explanation that directly reflects the evidence in the review.
</TASK>

<DECISION_PROCESS>
Follow this decision process internally before producing the final answer.

<STEP_1_OVERALL_DIRECTION>
First, determine the overall direction of the experience described in the review:
- Positive
- Neutral
- Negative
</STEP_1_OVERALL_DIRECTION>

<STEP_2_SIGNAL_EXTRACTION>
Identify concrete signals from the review:

<POSITIVE_SIGNALS>
Food quality, friendly or attentive service, good value, satisfaction, recommendations, intent to return.
</POSITIVE_SIGNALS>

<NEGATIVE_SIGNALS>
Long waits, being ignored, rude or inattentive staff, poor quality, price complaints, disappointment, wasted time.
</NEGATIVE_SIGNALS>
</STEP_2_SIGNAL_EXTRACTION>

<STEP_3_YELP_SPECIFIC_RULES>
Apply these Yelp-specific rules strictly:
- Service failures (e.g., being ignored, excessive wait times, inattentive staff) strongly cap the rating at 2.
- Statements implying regret, disappointment, wasted time, or not returning cap the rating at 3 or below.
- Price complaints significantly reduce ratings unless clearly offset by strong value.
- Ambiance or atmosphere alone cannot outweigh poor service or quality issues.
</STEP_3_YELP_SPECIFIC_RULES>

<STEP_4_RATING_SELECTION>
Based on the overall direction and applied rules, assign a single precise star rating (1-5) that best represents how a Yelp user would rate this experience.
</STEP_4_RATING_SELECTION>
</DECISION_PROCESS>

<RATING_CALIBRATION>
5: Strongly positive, enthusiastic, clear satisfaction and intent to return  
4: Mostly positive with minor issues  
3: Balanced positives and negatives or average experience  
2: Mostly negative with limited positives  
1: Extremely negative, severe dissatisfaction  
</RATING_CALIBRATION>

<SHORT_REVIEW_HANDLING>
If the review is very short or vague, rely on decisive keywords and implied intent rather than assumptions or inferred context.
</SHORT_REVIEW_HANDLING>

<OUTPUT_RULES>
- Follow {format_instructions} exactly.
- Output valid JSON only.
- Do not include extra fields, text, or commentary.
</OUTPUT_RULES>


"""),
("human","""Please analyze the following Yelp review and predict its star rating.

REVIEW:
{review_text}

---

{format_instructions}

Provide your prediction in the exact JSON format specified above.
""")])

prompt_template3 = ChatPromptTemplate.from_messages([
    ("system","""
# Role & Objective

You are an expert AI system that predicts **Yelp star ratings (1–5)** from user review text and provides a **brief, evidence-based explanation** grounded strictly in the review content.

Your goal is to replicate how a real Yelp user would assign a star rating.

---

# Core Instructions (Follow Exactly)

- Use **only the provided review text** as evidence.
- Do **not** rely on external knowledge, assumptions, or unstated context.
- Assign **one integer star rating only (1–5)**.
- Provide a **concise explanation** that explicitly references signals from the review.
- If information is insufficient, make a decision based on **explicit keywords and implied sentiment**, not speculation.

---

# Required Reasoning Workflow (Must Be Reflected in Output)

You must follow and **explicitly show** the following steps in order before finalizing the rating:

## Step 1: Determine Overall Sentiment Direction
Classify the experience as one of:
- **Positive**
- **Neutral**
- **Negative**

## Step 2: Extract Concrete Signals from the Review

### Positive Signals
- Food quality  
- Friendly or attentive service  
- Good value for money  
- Satisfaction or enjoyment  
- Recommendations  
- Intent to return  

### Negative Signals
- Long waits or delays  
- Being ignored  
- Rude or inattentive staff  
- Poor food or service quality  
- Price complaints  
- Disappointment, regret, or wasted time  

Only count signals that are **explicitly stated or strongly implied**.

## Step 3: Apply Yelp-Specific Rating Rules (Strict)

Apply these rules literally:

- **Service failures** (ignored, rude staff, excessive wait) → rating **cannot exceed 2**
- **Regret or “would not return” statements** → rating **cannot exceed 3**
- **Price complaints** → lower the rating unless clearly offset by strong value
- **Ambiance alone** cannot outweigh service or quality failures

If multiple rules apply, the **most restrictive rule wins**.

## Step 4: Select Final Rating
Choose the **single best integer rating (1–5)** that aligns with:
- The overall sentiment
- The extracted signals
- The Yelp-specific rules

---

# Rating Calibration Reference

Use this scale exactly:

- **5** — Strongly positive, enthusiastic, clear satisfaction and intent to return  
- **4** — Mostly positive with minor issues  
- **3** — Balanced positives and negatives, average experience  
- **2** — Mostly negative with limited positives  
- **1** — Extremely negative, severe dissatisfaction  

---

# Short or Ambiguous Reviews

If the review is **very short or vague**:
- Identify **high-impact keywords** (e.g., “great,” “terrible,” “never again”)
- Infer sentiment **only from those words**
- Avoid filling in missing details

---

# Output Format (Mandatory)

- Output **valid JSON only**
- Follow `{format_instructions}` **exactly**
- Do **not** include extra text, explanations, or fields outside the JSON

"""),

("human","""
Please analyze the following Yelp review and predict its star rating.

REVIEW:
{review_text}

{format_instructions}

Provide your prediction in the exact JSON format specified above.
""")
])


---
## 7. Chain Construction (Modern LCEL)

We build the pipeline using **LangChain Expression Language (LCEL)**:

```
prompt | llm | output_parser
```

This is the modern, recommended approach that:
- Avoids deprecated `LLMChain` classes
- Provides better composability
- Supports async, batching, and streaming out of the box

In [125]:
# Build the LCEL chain
chain1 = prompt_template1 | llm | output_parser
chain2 = prompt_template2 | llm | output_parser
chain3 = prompt_template3 | llm | output_parser


In [126]:
# Test the chain with a sample review
test_review = "The food was absolutely amazing! Best sushi I've ever had. Service was impeccable."

test_result = chain1.invoke({
    "review_text": test_review,
    "format_instructions": format_instructions
})

print("🧪 Test prediction:")
print(f"   Review: {test_review[:50]}...")
print(f"   Predicted stars: {test_result.predicted_stars}")
print(f"   Explanation: {test_result.explanation}")

🧪 Test prediction:
   Review: The food was absolutely amazing! Best sushi I've e...
   Predicted stars: 5
   Explanation: The review uses highly positive language, praising the food as 'absolutely amazing' and 'best sushi ever,' along with mentioning 'impeccable' service, indicating an excellent experience deserving of a 5-star rating.


---
## 8. Prediction Function

A reusable function that:
- Takes review text as input
- Returns the parsed prediction, JSON validity flag, and any errors
- Handles parsing errors and LLM failures gracefully

In [127]:
def predict_rating(
    review_text: str,
    chain: Any,
    format_instructions: str
) -> Tuple[Optional[RatingPrediction], bool, Optional[str]]:
    """
    Predict the star rating for a review.
    
    Args:
        review_text: The review text to analyze
        chain: The LCEL chain for prediction
        format_instructions: Format instructions for the LLM
    
    Returns:
        Tuple of:
        - prediction: RatingPrediction object (or None if failed)
        - json_valid: Whether the output was valid JSON
        - error: Error message (or None if successful)
    """
    try:
        # Invoke the chain
        prediction = chain.invoke({
            "review_text": review_text,
            "format_instructions": format_instructions
        })
        
        # If we get here, parsing was successful
        return prediction, True, None
        
    except Exception as e:
        error_msg = str(e)
        
        # Check if it's a parsing error vs LLM error
        if "validation error" in error_msg.lower() or "json" in error_msg.lower():
            return None, False, f"Parsing error: {error_msg[:200]}"
        else:
            return None, False, f"LLM error: {error_msg[:200]}"

print("✅ Prediction function defined")

✅ Prediction function defined


---
## 9. Batch Inference Loop

Run predictions on the entire sampled dataset, storing:
- Review ID and text
- Actual vs predicted stars
- Explanation
- JSON validity status

In [128]:
# Define chains and their names
chains = {
    "v1": chain1,  # Baseline prompt
    "v2": chain2,  # Improved prompt
    "v3": chain3   # Optimized prompt
}

# Store all results
all_results = {}

for prompt_version, chain in chains.items():
    results = []
    
    print(f"\n🚀 Running inference for Prompt {prompt_version.upper()} on {len(df_sample)} reviews...")
    print("=" * 60)
    
    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc=f"Prompt {prompt_version}"):
        review_id = row['review_id']
        review_text = row['text']
        actual_stars = row['stars']
        
        # Make prediction
        prediction, json_valid, error = predict_rating(
            review_text=review_text,
            chain=chain,
            format_instructions=format_instructions
        )
        
        # Store result
        result = {
            "review_id": review_id,
            "review_text": review_text[:500],
            "actual_stars": actual_stars,
            "predicted_stars": prediction.predicted_stars if prediction else None,
            "explanation": prediction.explanation if prediction else None,
            "json_valid": json_valid,
            "error": error
        }
        results.append(result)
    
    # Convert to DataFrame and store
    df_results = pd.DataFrame(results)
    all_results[prompt_version] = df_results
    
    # Save CSV and JSON for this prompt version
    csv_file = f"task1_prompt_{prompt_version}_results.csv"
    json_file = f"task1_prompt_{prompt_version}_results.json"
    
    df_results.to_csv(csv_file, index=False)
    
    # Create JSON in Fynd format
    json_output = [
        {
            "predicted_stars": int(row["predicted_stars"]) if pd.notna(row["predicted_stars"]) else None,
            "explanation": row["explanation"] if pd.notna(row["explanation"]) else None
        }
        for _, row in df_results.iterrows()
    ]
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(json_output, f, indent=2, ensure_ascii=False)
    
    # Print summary
    valid_df = df_results[df_results['json_valid']]
    accuracy = (valid_df['actual_stars'] == valid_df['predicted_stars']).mean() * 100 if len(valid_df) > 0 else 0
    json_rate = df_results['json_valid'].mean() * 100
    
    print(f"\n✅ Prompt {prompt_version.upper()} Complete")
    print(f"   Accuracy: {accuracy:.1f}%")
    print(f"   JSON Validity: {json_rate:.1f}%")
    print(f"   Saved: {csv_file}, {json_file}")

print("\n" + "=" * 60)
print("🎉 All prompts evaluated!")




🚀 Running inference for Prompt V1 on 200 reviews...


Prompt v1: 100%|██████████| 200/200 [03:36<00:00,  1.08s/it]



✅ Prompt V1 Complete
   Accuracy: 69.5%
   JSON Validity: 100.0%
   Saved: task1_prompt_v1_results.csv, task1_prompt_v1_results.json

🚀 Running inference for Prompt V2 on 200 reviews...


Prompt v2: 100%|██████████| 200/200 [03:38<00:00,  1.09s/it]



✅ Prompt V2 Complete
   Accuracy: 65.0%
   JSON Validity: 100.0%
   Saved: task1_prompt_v2_results.csv, task1_prompt_v2_results.json

🚀 Running inference for Prompt V3 on 200 reviews...


Prompt v3: 100%|██████████| 200/200 [04:07<00:00,  1.24s/it]


✅ Prompt V3 Complete
   Accuracy: 68.5%
   JSON Validity: 100.0%
   Saved: task1_prompt_v3_results.csv, task1_prompt_v3_results.json

🎉 All prompts evaluated!


---
## 10. Evaluation Metrics

Compute and display:
- **Accuracy**: Percentage of correct predictions (valid JSON only)
- **JSON Validity Rate**: Percentage of successfully parsed responses
- **Per-Star Metrics**: Accuracy breakdown by actual star rating

In [129]:
# Evaluation metrics for all prompts
print("📊 EVALUATION METRICS - ALL PROMPTS")
print("=" * 60)

# Store metrics for comparison table
metrics_summary = []

for prompt_version, df_results in all_results.items():
    print(f"\n{'─' * 60}")
    print(f"📌 PROMPT {prompt_version.upper()}")
    print(f"{'─' * 60}")
    
    # Filter to valid predictions only
    df_valid = df_results[df_results['json_valid']].copy()
    
    # JSON Validity Rate
    json_validity_rate = df_results['json_valid'].mean() * 100
    print(f"\n🔗 JSON Validity Rate: {json_validity_rate:.1f}%")
    print(f"   Valid: {df_results['json_valid'].sum()} / {len(df_results)}")
    
    # Overall Accuracy (valid predictions only)
    if len(df_valid) > 0:
        correct_predictions = (df_valid['actual_stars'] == df_valid['predicted_stars']).sum()
        accuracy = correct_predictions / len(df_valid) * 100
        print(f"\n🎯 Overall Accuracy: {accuracy:.1f}%")
        print(f"   Correct: {correct_predictions} / {len(df_valid)}")
    else:
        print("\n⚠️ No valid predictions to evaluate accuracy")
        accuracy = 0
    
    # Per-star accuracy
    print(f"\n📈 Per-Star Accuracy:")
    per_star_acc = {}
    for star in range(1, 6):
        star_df = df_valid[df_valid['actual_stars'] == star]
        if len(star_df) > 0:
            star_correct = (star_df['actual_stars'] == star_df['predicted_stars']).sum()
            star_accuracy = star_correct / len(star_df) * 100
            per_star_acc[star] = star_accuracy
            print(f"   {star}⭐: {star_accuracy:5.1f}% ({star_correct:2d} / {len(star_df):2d})")
        else:
            per_star_acc[star] = 0
            print(f"   {star}⭐: N/A (no samples)")
    
    # Store for comparison
    metrics_summary.append({
        "prompt": prompt_version.upper(),
        "accuracy": accuracy,
        "json_validity": json_validity_rate,
        "1_star_acc": per_star_acc.get(1, 0),
        "2_star_acc": per_star_acc.get(2, 0),
        "3_star_acc": per_star_acc.get(3, 0),
        "4_star_acc": per_star_acc.get(4, 0),
        "5_star_acc": per_star_acc.get(5, 0)
    })



📊 EVALUATION METRICS - ALL PROMPTS

────────────────────────────────────────────────────────────
📌 PROMPT V1
────────────────────────────────────────────────────────────

🔗 JSON Validity Rate: 100.0%
   Valid: 200 / 200

🎯 Overall Accuracy: 69.5%
   Correct: 139 / 200

📈 Per-Star Accuracy:
   1⭐:  70.0% (28 / 40)
   2⭐:  60.0% (24 / 40)
   3⭐:  67.5% (27 / 40)
   4⭐:  75.0% (30 / 40)
   5⭐:  75.0% (30 / 40)

────────────────────────────────────────────────────────────
📌 PROMPT V2
────────────────────────────────────────────────────────────

🔗 JSON Validity Rate: 100.0%
   Valid: 200 / 200

🎯 Overall Accuracy: 65.0%
   Correct: 130 / 200

📈 Per-Star Accuracy:
   1⭐:  55.0% (22 / 40)
   2⭐:  62.5% (25 / 40)
   3⭐:  67.5% (27 / 40)
   4⭐:  75.0% (30 / 40)
   5⭐:  65.0% (26 / 40)

────────────────────────────────────────────────────────────
📌 PROMPT V3
────────────────────────────────────────────────────────────

🔗 JSON Validity Rate: 100.0%
   Valid: 200 / 200

🎯 Overall Accuracy: 68.5%
 

In [130]:
# Create comparison DataFrame
df_comparison = pd.DataFrame(metrics_summary)
print("\n" + "=" * 60)
print("📊 COMPARISON TABLE")
print("=" * 60)
print(df_comparison.to_string(index=False))



📊 COMPARISON TABLE
prompt  accuracy  json_validity  1_star_acc  2_star_acc  3_star_acc  4_star_acc  5_star_acc
    V1      69.5          100.0        70.0        60.0        67.5        75.0        75.0
    V2      65.0          100.0        55.0        62.5        67.5        75.0        65.0
    V3      68.5          100.0        67.5        62.5        70.0        70.0        72.5


In [131]:
# Per-star accuracy breakdown for all prompts
print("\n📈 Per-Star Accuracy Breakdown")
print("=" * 60)

for prompt_version, df_results in all_results.items():
    df_valid = df_results[df_results['json_valid']].copy()
    
    print(f"\n📌 Prompt {prompt_version.upper()}:")
    print("-" * 40)
    
    for star in range(1, 6):
        star_df = df_valid[df_valid['actual_stars'] == star]
        if len(star_df) > 0:
            star_correct = (star_df['actual_stars'] == star_df['predicted_stars']).sum()
            star_accuracy = star_correct / len(star_df) * 100
            print(f"   {star}⭐: {star_accuracy:5.1f}% ({star_correct:2d} / {len(star_df):2d})")
        else:
            print(f"   {star}⭐: N/A (no samples)")




📈 Per-Star Accuracy Breakdown

📌 Prompt V1:
----------------------------------------
   1⭐:  70.0% (28 / 40)
   2⭐:  60.0% (24 / 40)
   3⭐:  67.5% (27 / 40)
   4⭐:  75.0% (30 / 40)
   5⭐:  75.0% (30 / 40)

📌 Prompt V2:
----------------------------------------
   1⭐:  55.0% (22 / 40)
   2⭐:  62.5% (25 / 40)
   3⭐:  67.5% (27 / 40)
   4⭐:  75.0% (30 / 40)
   5⭐:  65.0% (26 / 40)

📌 Prompt V3:
----------------------------------------
   1⭐:  67.5% (27 / 40)
   2⭐:  62.5% (25 / 40)
   3⭐:  70.0% (28 / 40)
   4⭐:  70.0% (28 / 40)
   5⭐:  72.5% (29 / 40)


In [132]:
# Prediction distribution for all prompts
print("\n📊 Prediction Distribution - All Prompts")
print("=" * 60)

for prompt_version, df_results in all_results.items():
    df_valid = df_results[df_results['json_valid']].copy()
    
    print(f"\n📌 Prompt {prompt_version.upper()}:")
    print("-" * 40)
    
    if len(df_valid) > 0:
        pred_counts = df_valid['predicted_stars'].value_counts().sort_index()
        actual_counts = df_valid['actual_stars'].value_counts().sort_index()
        
        print(f"{'Star':<6} {'Actual':<10} {'Predicted':<10}")
        print("-" * 26)
        for star in range(1, 6):
            actual = actual_counts.get(star, 0)
            predicted = pred_counts.get(star, 0)
            print(f"{star}⭐    {actual:<10} {predicted:<10}")




📊 Prediction Distribution - All Prompts

📌 Prompt V1:
----------------------------------------
Star   Actual     Predicted 
--------------------------
1⭐    40         36        
2⭐    40         42        
3⭐    40         40        
4⭐    40         45        
5⭐    40         37        

📌 Prompt V2:
----------------------------------------
Star   Actual     Predicted 
--------------------------
1⭐    40         30        
2⭐    40         50        
3⭐    40         44        
4⭐    40         47        
5⭐    40         29        

📌 Prompt V3:
----------------------------------------
Star   Actual     Predicted 
--------------------------
1⭐    40         35        
2⭐    40         44        
3⭐    40         44        
4⭐    40         42        
5⭐    40         35        


In [133]:
# Confusion matrix for all prompts
print("\n📋 Confusion Matrices (Actual vs Predicted)")
print("=" * 60)

for prompt_version, df_results in all_results.items():
    df_valid = df_results[df_results['json_valid']].copy()
    
    print(f"\n📌 Prompt {prompt_version.upper()}:")
    print("-" * 50)
    
    if len(df_valid) > 0:
        confusion = pd.crosstab(
            df_valid['actual_stars'], 
            df_valid['predicted_stars'],
            rownames=['Actual'],
            colnames=['Predicted']
        )
        print(confusion)
    else:
        print("No valid predictions for confusion matrix")




📋 Confusion Matrices (Actual vs Predicted)

📌 Prompt V1:
--------------------------------------------------
Predicted   1   2   3   4   5
Actual                       
1          28  12   0   0   0
2           7  24   9   0   0
3           0   6  27   7   0
4           0   0   3  30   7
5           1   0   1   8  30

📌 Prompt V2:
--------------------------------------------------
Predicted   1   2   3   4   5
Actual                       
1          22  18   0   0   0
2           7  25   8   0   0
3           0   7  27   6   0
4           0   0   7  30   3
5           1   0   2  11  26

📌 Prompt V3:
--------------------------------------------------
Predicted   1   2   3   4   5
Actual                       
1          27  13   0   0   0
2           7  25   8   0   0
3           0   6  28   6   0
4           0   0   6  28   6
5           1   0   2   8  29
